In [131]:
import pandas as pd
import os

In [386]:
asv_file = pd.read_csv("vsearch_dada2_gtdb_ure_merged.tsv", sep = "\t")
contribution_ko_file = pd.read_csv("KO_metagenome_out/pred_metagenome_contrib.tsv", sep = '\t')
filtered_ko = contribution_ko_file[contribution_ko_file['function'].str.contains('21463')]

In [387]:
metadata_file = pd.read_csv("w9w10_subset_metadata.csv")
taxon_info = filtered_ko['taxon'].tolist()
unique_mice = filtered_ko['sample'].nunique()
#unique_mice # prints the number of unique mice found for that particular KO/EC/pathway, regardless of the group it belongs to
taxon_unique = filtered_ko['taxon'].nunique()
#taxon_unique #printes the number of unique taxon (asvs) linked to that particular KO/EC/Pathway, regardless of the grou it belongs to
asv_file = asv_file.rename(columns={'asv_num': 'taxon'}) #renaming the ASV file so we can merge them with the picrust2 output
asv_file_filtered = asv_file[['taxon','family_final','genus_final','species_final']]
merged_df = pd.merge(filtered_ko, asv_file_filtered, on = 'taxon', how='left')
merged_df['sample'] = merged_df['sample'].str.lstrip('X') #Removing the annoying Xs that Excel adds to my file
metadata_file = metadata_file.rename(columns={'Sequence ID':'sample'}) #Renaming it so it can be merged
metadata_file_filtered = metadata_file[['sample','Diabetic','FatePrimary','Sex','Human_derived_status']] #including the metadata values
merged_df_final = pd.merge(merged_df, metadata_file_filtered, on='sample', how='left') #merging the metadata file into the previously merged file
merged_df_final['genus_species'] = merged_df_final['genus_final']+"_"+merged_df_final['species_final'] #merging genus and species in one column
merged_df_final['Human_derived_status'] = merged_df_final['Human_derived_status'].replace("IAB+","seroconverted")
merged_df_final['Human_derived_status'] = merged_df_final['Human_derived_status'].replace("IAB-","negative")

In [388]:
#Getting the number of genomes and types per KO of interest
merged_df_final_s2 = merged_df_final[merged_df_final['Human_derived_status'].str.contains('negative')]
unique_species = merged_df_final_s2['genus_species'].unique()
unique_genus = merged_df_final_s2['genus_final'].unique()
print(unique_species)
unique_mice = merged_df_final_s2['sample'].nunique()
print(unique_mice)
print(unique_genus)
merged_df_final_s2

['Staphylococcus_hominis' nan 'Enterococcus_faecalis']
8
['Staphylococcus' nan 'Enterococcus']


,sample,function,taxon,taxon_abun,taxon_rel_abun,genome_function_count,taxon_function_abun,taxon_rel_function_abun,norm_taxon_function_contrib,family_final,genus_final,species_final,Diabetic,FatePrimary,Sex,Human_derived_status,genus_species
0,3975R_L_F_NS1_week9_S82_L001,K21463,ASV_1034,3.33,0.022835,1,3.33,0.022835,1.000,Staphylococcaceae,Staphylococcus,hominis,yes,NS1,Female,negative,Staphylococcus_hominis
13,2808_NS1_RR_Female_week9_S52_L001,K21463,ASV_1343,1.00,0.004487,1,1.00,0.004487,1.000,Enterococcaceae,NaN,NaN,no,NS1,Female,negative,NaN
32,2562R_0_F_NS6_week9_S82_L001,K21463,ASV_1021,6.00,0.031465,1,6.00,0.031465,1.000,Enterococcaceae,Enterococcus,faecalis,yes,NS6,Female,negative,Enterococcus_faecalis
182,2824_NS1_0_Male_week9_S55_L001,K21463,ASV_1816,0.40,0.001410,1,0.40,0.001410,0.500,Enterococcaceae,NaN,NaN,no,NS1,Male,negative,NaN
183,2824_NS1_0_Male_week9_S55_L001,K21463,ASV_1820,0.40,0.001410,1,0.40,0.001410,0.500,Enterococcaceae,NaN,NaN,no,NS1,Male,negative,NaN
244,1030R_L_M_NS1_week9_S25_L001,K21463,ASV_1623,0.75,0.004199,1,0.75,0.004199,0.375,Enterococcaceae,NaN,NaN,no,NS1,Male,negative,NaN
245,1030R_L_M_NS1_week9_S25_L001,K21463,ASV_1624,0.75,0.004199,1,0.75,0.004199,0.375,Enterococcaceae,NaN,NaN,no,NS1,Male,negative,NaN
246,1030R_L_M_NS1_week9_S25_L001,K21463,ASV_1832,0.50,0.002799,1,0.50,0.002799,0.250,Enterococcaceae,NaN,NaN,no,NS1,Male,negative,NaN
308,1121R_0_M_NS1_week9_S29_L001,K21463,ASV_1260,1.40,0.008500,1,1.40,0.008500,1.000,Enterococcaceae,NaN,NaN,no,NS1,Male,negative,NaN
382,1186R_0_M_NS6_week9_S86_L001,K21463,ASV_1137,2.75,0.013057,1,2.75,0.013057,1.000,Enterococcaceae,Enterococcus,faecalis,yes,NS6,Male,negative,Enterococcus_faecalis


In [389]:
merged_df_final_ns1 = merged_df_final[merged_df_final['Human_derived_status'].str.contains('seroconverted')]
unique_species = merged_df_final_ns1['genus_species'].unique()
print(unique_species)
unique_mice = merged_df_final_ns1['sample'].nunique()
unique_genus = merged_df_final_ns1['genus_final'].unique()
print(unique_mice)
print(unique_genus)
merged_df_final_ns1

[nan 'Enterococcus_A_hulanensis/hermanniensis/avium/raffinosus'
 'Enterococcus_A_hulanensis/hermanniensis/raffinosus'
 'Enterococcus_A_avium']
70
['Enterococcus_A' nan 'Staphylococcus']


,sample,function,taxon,taxon_abun,taxon_rel_abun,genome_function_count,taxon_function_abun,taxon_rel_function_abun,norm_taxon_function_contrib,family_final,genus_final,species_final,Diabetic,FatePrimary,Sex,Human_derived_status,genus_species
1,2433R_0_F_S5_week9_S71_L001,K21463,ASV_1044,1.2,0.005992,1,1.2,0.005992,0.005128,Enterococcaceae,Enterococcus_A,NaN,yes,S5,Female,seroconverted,NaN
2,2433R_0_F_S5_week9_S71_L001,K21463,ASV_1426,0.8,0.003995,1,0.8,0.003995,0.003419,Enterococcaceae,Enterococcus_A,NaN,yes,S5,Female,seroconverted,NaN
3,2433R_0_F_S5_week9_S71_L001,K21463,ASV_236,107.8,0.538272,1,107.8,0.538272,0.460684,Enterococcaceae,Enterococcus_A,hulanensis/hermanniensis/avium/raffinosus,yes,S5,Female,seroconverted,Enterococcus_A_hulanensis/hermanniensis/avium/...
4,2433R_0_F_S5_week9_S71_L001,K21463,ASV_493,28.4,0.141808,1,28.4,0.141808,0.121368,Enterococcaceae,Enterococcus_A,hulanensis/hermanniensis/avium/raffinosus,yes,S5,Female,seroconverted,Enterococcus_A_hulanensis/hermanniensis/avium/...
5,2433R_0_F_S5_week9_S71_L001,K21463,ASV_501,25.6,0.127827,1,25.6,0.127827,0.109402,Enterococcaceae,Enterococcus_A,NaN,yes,S5,Female,seroconverted,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,1152R_R_M_S2_week9_S65_L001,K21463,ASV_422,23.6,0.181934,1,23.6,0.181934,0.384365,Enterococcaceae,Enterococcus_A,avium,no,S2,Male,seroconverted,Enterococcus_A_avium
463,3924R_L_F_S5_week9_S72_L001,K21463,ASV_353,44.2,0.223228,1,44.2,0.223228,0.413858,Enterococcaceae,Enterococcus_A,hulanensis/hermanniensis/avium/raffinosus,yes,S5,Female,seroconverted,Enterococcus_A_hulanensis/hermanniensis/avium/...
464,3924R_L_F_S5_week9_S72_L001,K21463,ASV_505,51.0,0.257571,1,51.0,0.257571,0.477528,Enterococcaceae,Enterococcus_A,hulanensis/hermanniensis/avium/raffinosus,yes,S5,Female,seroconverted,Enterococcus_A_hulanensis/hermanniensis/avium/...
465,3924R_L_F_S5_week9_S72_L001,K21463,ASV_589,8.4,0.042423,1,8.4,0.042423,0.078652,Enterococcaceae,Enterococcus_A,hulanensis/hermanniensis/raffinosus,yes,S5,Female,seroconverted,Enterococcus_A_hulanensis/hermanniensis/raffin...
